In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,LabelEncoder,RobustScaler,LabelEncoder,TargetEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor

In [2]:
df=pd.read_csv(r'C:\Users\pavan\Desktop\data science\KNN\diamonds.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  53940 non-null  int64  
 1   carat       53940 non-null  float64
 2   cut         53940 non-null  object 
 3   color       53940 non-null  object 
 4   clarity     53940 non-null  object 
 5   depth       53940 non-null  float64
 6   table       53940 non-null  float64
 7   price       53940 non-null  int64  
 8   x           53940 non-null  float64
 9   y           53940 non-null  float64
 10  z           53940 non-null  float64
dtypes: float64(6), int64(2), object(3)
memory usage: 4.5+ MB


In [4]:
df.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [5]:
df=df.drop(columns='Unnamed: 0')

In [6]:
cat=df.select_dtypes(exclude='number')

In [7]:
for col in cat.columns:
    print(col,'-->',cat[col].nunique())

cut --> 5
color --> 7
clarity --> 8


In [8]:
x=df.drop(columns='price')
y=df['price']

In [9]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,random_state=42)

In [10]:
num=x_train.select_dtypes(include='number').columns
cat=x_train.select_dtypes(exclude='number').columns

In [11]:
num_pipleline=Pipeline(
    steps=[
        ('num_scaling',RobustScaler())
    ]
)


cat_pipleline=Pipeline(
    steps=[
        ('car_encoding',TargetEncoder())
    ]
)

In [12]:
preprocessing=ColumnTransformer(
    transformers=[
        ('num_pipleline',num_pipleline,num),
        ('cat_pipleline',cat_pipleline,cat)
    ]
)

In [13]:
main_pipeline=Pipeline(
    steps=[
        ("preprocessing",preprocessing),
        ('model',KNeighborsRegressor(metric='manhattan',n_neighbors=5))
    ]
)

In [14]:
gridsearch = GridSearchCV(
    estimator=main_pipeline,
    param_grid={
        'model__n_neighbors': [3, 5, 7, 9, 11],
        'model__weights': ['uniform', 'distance'],
        'model__metric': ['euclidean', 'manhattan', 'minkowski'],
        'model__p': [1, 2]   # p=1 → Manhattan, p=2 → Euclidean
    },
    cv=3,
    verbose=3,
    n_jobs=-1,
    scoring='neg_mean_absolute_error'
)

In [ ]:
gridsearch.fit(x_train,y_train)

Fitting 3 folds for each of 60 candidates, totalling 180 fits
